# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [57]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-02-01	Evolution of dark matter haloes in CIELO simulations   	BAAA64	This article analyze the dark matter (dm) halo evolution through time. For this study we use a Milky Way like haloes subsample from CIELO (ChemodynamIcal propertiEs of gaLaxies and the cOsmic web) project. This project aims at studying the formation of galaxies in different environments using zoom-ins simulations.	Cataldi, et. al. (2023b)	Evolution of dark matter haloes in CIELO simulations   	http://astronomiaargentina.fcaglp.unlp.edu.ar/article/printPreview/id/39
2023-05-01	Redshift evolution of the dark matter haloes shapes	MNRAS	In this work, we aim at investigating the morphology evolution of Milky Way mass-like dark matter haloes selected from the CIELO and IllustrisTNG Projects. The connection between halo shapes and their environment has been studied in previous works at z=0 but their connection remains yet to be fully understood. We focus on the ev

## Import pandas

We are using the very handy pandas library for dataframes.

In [58]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [59]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-02-01,Evolution of dark matter haloes in CIELO simul...,BAAA64,This article analyze the dark matter (dm) halo...,"Cataldi, et. al. (2023b)",Evolution of dark matter haloes in CIELO simul...,http://astronomiaargentina.fcaglp.unlp.edu.ar/...
1,2023-05-01,Redshift evolution of the dark matter haloes s...,MNRAS,"In this work, we aim at investigating the morp...","Cataldi, et. al. (2023a)",Redshift evolution of the dark matter haloes s...,https://doi.org/10.1093/mnras/stad1601
2,2022-10-01,Fingerprints of modified gravity on galaxies i...,MNRAS,"In this work, we search for detectable signatu...","Cataldi, et. al. (2022)",Fingerprints of modified gravity on galaxies i...,https://doi.org/10.1093/mnras/stac2122
3,2020-12-01,Baryons shaping dark matter haloes,MNRAS,In this work we aim at investigating the effec...,"Cataldi, et. al. (2020)",Baryons shaping dark matter haloes,http://dx.doi.org/10.1093/mnras/staa3988
4,2019-02-01,Dark matter response to galaxy assembly history,A&A,"In this article, we study the properties of fo...","Artale, et. al. (2019)",Dark matter response to galaxy assembly history,http://dx.doi.org/10.1051/0004-6361/201834096


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [60]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [61]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [62]:
!ls ../_publications/

'2019-02-01-Dark matter response to galaxy assembly history .md'
'2020-12-01-Baryons shaping dark matter haloes  .md'
'2022-10-01-Fingerprints of modified gravity on galaxies in voids.md'
'2023-02-01-Evolution of dark matter haloes in CIELO simulations   .md'
'2023-05-01-Redshift evolution of the dark matter haloes shapes.md'


In [63]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
